In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2).json


{'kaggle (2).json': b'{"username":"mahajanbhushan","key":"88002547ce5a2790b7eda0352585b43c"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp "kaggle (2).json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balraj98/deepglobe-road-extraction-dataset
!unzip -q deepglobe-road-extraction-dataset.zip -d dataset-folder

100% 3.79G/3.79G [03:04<00:00, 22.9MB/s]
100% 3.79G/3.79G [03:04<00:00, 22.0MB/s]


In [ ]:
import os
import cv2
import numpy as np
dataset_folder = '/content/dataset-folder/train'
images = []
masks = []
file_list = os.listdir(dataset_folder)
file_list.sort()
for i in range(0, len(file_list), 2):
    satellite_image_filename = file_list[i + 1]
    mask_image_filename = file_list[i]
    satellite_image_path = os.path.join(dataset_folder, satellite_image_filename)
    mask_image_path = os.path.join(dataset_folder, mask_image_filename)
    satellite_image = cv2.imread(satellite_image_path)
    satellite_image = cv2.cvtColor(satellite_image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    satellite_image = cv2.resize(satellite_image, (128, 128))  # Resize as (128,128)
    satellite_image = satellite_image / 255.0  # Normalize pixel values to [0, 1]
    mask_image = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)
    mask_image = cv2.resize(mask_image, (128, 128))  # Resize to (128,128)
    mask_image = np.expand_dims(mask_image, axis=-1)  # Adding a channel dimension
    mask_image = mask_image / 255.0
    images.append(satellite_image)
    masks.append(mask_image)
images = np.array(images)
masks = np.array(masks)#Converting to array

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    images,
    masks,
    test_size=0.1,
    random_state=42
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_image_data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_mask_data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=lambda x: np.where(x > 0.5, 1, 0).astype(x.dtype)
)

In [ ]:
seed = 42
train_image_generator = train_image_data_gen.flow(X_train, seed=seed, batch_size=32)
train_mask_generator = train_mask_data_gen.flow(y_train, seed=seed, batch_size=32)
train_generator = zip(train_image_generator, train_mask_generator)

In [ ]:
valid_image_data_gen = ImageDataGenerator()
valid_mask_data_gen = ImageDataGenerator()
valid_image_generator = valid_image_data_gen.flow(X_valid, batch_size=32, seed=seed, shuffle=False)
valid_mask_generator = valid_mask_data_gen.flow(y_valid, batch_size=32, seed=seed, shuffle=False)
valid_generator = zip(valid_image_generator, valid_mask_generator)

In [ ]:
from keras.applications import ResNet50, VGG16
from keras.layers import Input, Concatenate, Conv2D, UpSampling2D
from keras.models import Model
input_tensor = Input(shape=(128, 128, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
resnet_features = resnet_model(input_tensor)
vgg_features = vgg_model(input_tensor)
merged_features = Concatenate()([resnet_features, vgg_features])
x = Conv2D(64, (3, 3), activation='relu', padding='same')(merged_features)
output = UpSampling2D(size=(32, 32))(x)
output = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(output)

fusion_model = Model(inputs=input_tensor, outputs=output)
fusion_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
history = fusion_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_data=valid_generator,
    validation_steps=len(X_valid) // 32,
    epochs=10
)

58889256/58889256 [==============================] - 4s 0us/step
Epoch 1/10
175/175 [==============================] - 92s 268ms/step - loss: 0.3961 - accuracy: 0.9562 - val_loss: 0.1868 - val_accuracy: 0.9529
Epoch 2/10
175/175 [==============================] - 52s 283ms/step - loss: 0.1535 - accuracy: 0.9584 - val_loss: 0.3041 - val_accuracy: 0.9527
Epoch 3/10
175/175 [==============================] - 48s 276ms/step - loss: 0.1504 - accuracy: 0.9585 - val_loss: 0.2188 - val_accuracy: 0.9524
Epoch 4/10
175/175 [==============================] - 48s 275ms/step - loss: 0.1498 - accuracy: 0.9584 - val_loss: 0.2591 - val_accuracy: 0.9525
Epoch 5/10
175/175 [==============================] - 48s 276ms/step - loss: 0.1485 - accuracy: 0.9582 - val_loss: 0.2452 - val_accuracy: 0.9523
Epoch 6/10
175/175 [==============================] - 48s 275ms/step - loss: 0.1469 - accuracy: 0.9585 - val_loss: 0.1781 - val_accuracy: 0.9520
Epoch 7/10
175/175 [==============================] - 48s 275ms/s

In [ ]:
import pickle
fusion_model_augmentation_file = "fusion_model_augmentation_model.pkl"
with open(fusion_model_augmentation_file, 'wb') as file:
    pickle.dump(fusion_model, file)

In [ ]:
import pickle
with open('/content/fusion_model_augmentation_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)
print("Training history saved!")

Training history saved!


In [ ]:
with open(model_pkl_file, 'rb') as file:
    model = pickle.load(file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.models import Model

def unet(input_size=(128, 128, 3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    conv1 = Dropout(0.1)(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.1)(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    conv3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    conv4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Dropout(0.3)(conv5)
    conv5 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    # Decoder
    decoder6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    decoder6 = concatenate([decoder6, conv4])
    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder6)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)
    decoder7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    decoder7 = concatenate([decoder7, conv3])
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder7)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    decoder8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    decoder8 = concatenate([decoder8, conv2])
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder8)
    conv8 = Dropout(0.1)(conv8)
    conv8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8)
    decoder9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    decoder9 = concatenate([decoder9, conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(decoder9)
    conv9 = Dropout(0.1)(conv9)
    conv9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9)
    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
unet_model = unet()
unet_model.summary()

In [ ]:
# Fit the model
history1 = unet_model.fit(
    train_generator,
    validation_data=valid_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_steps=len(X_valid) // 32,
    epochs=10
)

Epoch 1/10
175/175 [==============================] - 98s 480ms/step - loss: 0.2318 - accuracy: 0.9554 - val_loss: 0.1718 - val_accuracy: 0.9526
Epoch 2/10
175/175 [==============================] - 80s 456ms/step - loss: 0.1530 - accuracy: 0.9583 - val_loss: 0.1513 - val_accuracy: 0.9531
Epoch 3/10
175/175 [==============================] - 80s 456ms/step - loss: 0.1473 - accuracy: 0.9585 - val_loss: 0.1611 - val_accuracy: 0.9526
Epoch 4/10
175/175 [==============================] - 80s 457ms/step - loss: 0.1418 - accuracy: 0.9587 - val_loss: 0.1487 - val_accuracy: 0.9527
Epoch 5/10
175/175 [==============================] - 80s 457ms/step - loss: 0.1384 - accuracy: 0.9584 - val_loss: 0.1342 - val_accuracy: 0.9530
Epoch 6/10
175/175 [==============================] - 80s 458ms/step - loss: 0.1304 - accuracy: 0.9596 - val_loss: 0.1255 - val_accuracy: 0.9551
Epoch 7/10
175/175 [==============================] - 80s 459ms/step - loss: 0.1260 - accuracy: 0.9595 - val_loss: 0.1204 - val_ac

In [ ]:
import pickle
unet_model_augmentation_file = "/content/drive/MyDrive/unet_model_augmentation_model.pkl"
with open(unet_model_augmentation_file, 'wb') as file:
    pickle.dump(unet_model, file)

In [ ]:
import pickle
with open('/content/unet_model_augmentation_history.pkl', 'wb') as file:
    pickle.dump(history1.history, file)
print("Training history saved!")

Training history saved!
